Обучить полносвязную модель на MNIST.

Обучить глубокую сверточную сеть на MNIST.

*Обучить с нуля глубокую сеть на небольшом датасете. Сравнить результат с дообучением предобученной сети.

In [2]:
!pip install torchvision

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Обучение полносвязной сети на MNIST

In [4]:
# hyper parameters
input_size = 784 # 28x28
hidden_size = 100
num_classes = 10
num_epochs = 4
batch_size = 100
learning_rate = 0.001


# MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
n_classes = len(classes)


# nn
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        return out 
    
model = NeuralNet(input_size, hidden_size, num_classes)


# loss and optimiser
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)


# train
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backwards
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')
    
    
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # max returns (value, index)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc} %\n')
    
    for i in range(n_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {classes[i]}: {acc:.2f} %')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

epoch 1 / 4, step 100 / 600, loss = 1.0457
epoch 1 / 4, step 200 / 600, loss = 1.0837
epoch 1 / 4, step 300 / 600, loss = 1.0526
epoch 1 / 4, step 400 / 600, loss = 0.8962
epoch 1 / 4, step 500 / 600, loss = 1.0875
epoch 1 / 4, step 600 / 600, loss = 1.0949
epoch 2 / 4, step 100 / 600, loss = 0.8431
epoch 2 / 4, step 200 / 600, loss = 0.9403
epoch 2 / 4, step 300 / 600, loss = 0.8881
epoch 2 / 4, step 400 / 600, loss = 0.9172
epoch 2 / 4, step 500 / 600, loss = 0.8443
epoch 2 / 4, step 600 / 600, loss = 0.8965
epoch 3 / 4, step 100 / 600, loss = 0.8895
epoch 3 / 4, step 200 / 600, loss = 0.7740
epoch 3 / 4, step 300 / 600, loss = 0.9590
epoch 3 / 4, step 400 / 600, loss = 0.7906
epoch 3 / 4, step 500 / 600, loss = 0.7024
epoch 3 / 4, step 600 / 600, loss = 0.8421
epoch 4 / 4, step 100 / 600, loss = 0.7703
epoch 4 / 4, step 200 / 600, loss = 0.9224
epoch 4 / 4, step 300 / 600, loss = 0.9428
epoch 4 / 4, step 400 

# Обучение сверточной сети на MNIST

In [5]:
# hyper parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001


# MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
n_classes = len(classes)


# nn
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet().to(device)


# loss and optimiser
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)


# train
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backwards
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if (i+1) % 2000 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')


# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    
    for images, labels in test_loader:
        labels = labels.to(device)
        outputs = model(images)
        
        # max returns (value, index)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc} %\n')
    
    for i in range(n_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {classes[i]}: {acc:.2f} %')

epoch 1 / 4, step 2000 / 15000, loss = 2.3078
epoch 1 / 4, step 4000 / 15000, loss = 2.3131
epoch 1 / 4, step 6000 / 15000, loss = 2.2610
epoch 1 / 4, step 8000 / 15000, loss = 2.2908
epoch 1 / 4, step 10000 / 15000, loss = 2.2062
epoch 1 / 4, step 12000 / 15000, loss = 0.9868
epoch 1 / 4, step 14000 / 15000, loss = 0.5448
epoch 2 / 4, step 2000 / 15000, loss = 0.3413
epoch 2 / 4, step 4000 / 15000, loss = 0.3989
epoch 2 / 4, step 6000 / 15000, loss = 0.3515
epoch 2 / 4, step 8000 / 15000, loss = 0.2581
epoch 2 / 4, step 10000 / 15000, loss = 0.0242
epoch 2 / 4, step 12000 / 15000, loss = 0.0628
epoch 2 / 4, step 14000 / 15000, loss = 0.0119
epoch 3 / 4, step 2000 / 15000, loss = 0.0251
epoch 3 / 4, step 4000 / 15000, loss = 0.0320
epoch 3 / 4, step 6000 / 15000, loss = 0.0019
epoch 3 / 4, step 8000 / 15000, loss = 0.0089
epoch 3 / 4, step 10000 / 15000, loss = 0.0233
epoch 3 / 4, step 12000 / 15000, loss = 0.0109
epoch 3 / 4, step 14000 / 15000, loss = 0.0224
epoch 4 / 4, step 2000 / 